In [38]:
# url = 'https://www.booking.com/searchresults.en-us.html?ss={}&ssne={}&ssne_untouched={}&label=gen173nr-1FCAEoggI46AdIM1gEaKcCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuALAzp6cBsACAdICJDFhMjc3YWQ4LTY3ZjktNDNhNy05NTI0LTUxMWY3NWQ2ODFmNdgCBeACAQ&sid=2126271f814d7c3f7af3ef0e5ea6d865&aid=304142&lang=en-us&sb=1&src_elem=sb&src=index&dest_id=2469&dest_type=region&checkin={}&checkout={}&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'

In [39]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver.common.by import By
import random

In [40]:
from selenium.webdriver.common.keys import Keys

In [41]:
from selenium.webdriver import Chrome

In [42]:
path = 'C:\\BrowersDriver\\chromedriver.exe'

In [43]:
browser = Chrome(path)

C:\Users\mihir\AppData\Local\Temp\ipykernel_5240\2444750185.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(path)


In [44]:
url = "https://www.booking.com"

In [45]:
browser.get(url)

In [46]:
state_name=browser.find_element(By.NAME, "ss")

In [47]:
state_name.send_keys('Boston')

In [48]:
dates = browser.find_element(By.XPATH, "//div[@class='xp__dates-inner']")

In [49]:
dates.click()

In [50]:
Month_next = browser.find_element(By.XPATH,"//div[@data-bui-ref='calendar-next']")

In [51]:
Month_next.click()

In [52]:
check_in = browser.find_element(By.XPATH,"//td[@data-date='2023-02-08']")

In [53]:
check_out = browser.find_element(By.XPATH,"//td[@data-date='2023-02-14']")

In [54]:
check_in.click()

In [55]:
check_out.click()

In [56]:
submitbutton = browser.find_element(By.XPATH, "//button[@type='submit']")

In [57]:
submitbutton.click()

In [58]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [59]:
total_pages = int(browser.find_element(By.CSS_SELECTOR, 'div[data-testid="pagination"]  li:last-child').text)
# print(f"Total pages: {total_pages}")

In [60]:
all_detail_links = []

In [61]:
def get_links():
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    all_properties = soup.findAll("div", {"data-testid" : "property-card"})
    for property in all_properties:
        h3 = property.find('h3')
        link = h3.find('a')['href']
        all_detail_links.append(link)

In [62]:
import time

In [63]:
for current_page in range(1,total_pages+1):
    get_links()
    next_page_btn = browser.find_element(By.XPATH, '//button[contains(@aria-label, "Next page")]')
    next_page_btn.click()
    time.sleep(6)

In [64]:
import json

# create the scraped_hotels folder if you can create different different folder for different state name it would be great

In [65]:
def save_file(data,c):
    with open(f'Boston/{c}.json', 'w+') as f:
        json.dump(data, f, indent=4)

In [66]:
all_properties = []
c = 0
for link in all_detail_links:
    browser.get(link)
    c += 1
    try:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        hotel_label = soup.find('h2',class_ = 'pp-header__title').text
        location = soup.find('span',class_ = 'hp_address_subtitle').text.strip("\n")
        #total_images = soup.find('span', class_="bh-photo-grid-thumb-more").text
        rating_block = soup.find("div", {"data-testid" : "review-score-right-component"})
        rating_score = rating_block.findChild().text
        sub_block = rating_block.findChildren()[1]
        rating_label = sub_block.findChild().text
        total_reviews = sub_block.findChildren()[1].text
        best_facility = soup.find('p',class_ = 'best-review-score-label').text.strip("\n")
        best_review_score_badge = soup.find('span',class_='review-score-badge').text.strip("\n")
        detail = soup.find('div',class_ = 'hp_desc_main_content').text.replace("\n", "")
        couple_review = soup.find('p',class_ = 'hp-desc-review-highlight').text.strip("\n")
        important_facility = soup.find('div',class_ = 'hp_desc_important_facilities').text.replace("\n", "")
        price = soup.find('span',class_ = 'prco-valign-middle-helper').text.strip("\n")
    except:
        print(c,link)
        continue
    data = dict(hotel_label=hotel_label,
                location=location,
                #total_images=total_images,
                rating_score=rating_score,
                rating_label=rating_label,
                total_reviews=total_reviews,
                best_facility=best_facility,
                best_review_score_badge=best_review_score_badge,
                detail=detail,
                couple_review=couple_review,
                important_facility=important_facility,
                price=price,
               source=str(soup))
    save_file(data, c)
    time.sleep(random.randint(3,5))

16 https://www.booking.com/hotel/us/hampton-inn-logan-air.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=16&hapos=16&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794495&all_sr_blocks=2870615_344383329_2_1_0&highlighted_blocks=2870615_344383329_2_1_0&matching_block_id=2870615_344383329_2_1_0&sr_pri_blocks=2870615_344383329_2_1_0__91201&from=searchresults&map_fdco=1#hotelTmpl
26 https://www.booking.com/hotel/us/chic-spacious-2br-on-washington-st-by-stay-gia.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout

100 https://www.booking.com/hotel/us/comfy-beacon-hill-studio-great-for-work-travel-7.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=100&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794516&all_sr_blocks=489604201_361933709_2_0_0&highlighted_blocks=489604201_361933709_2_0_0&matching_block_id=489604201_361933709_2_0_0&sr_pri_blocks=489604201_361933709_2_0_0__97600&from=searchresults&map_fdco=1#hotelTmpl
101 https://www.booking.com/hotel/us/global-luxury-suites-east-boston.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&che

137 https://www.booking.com/hotel/us/courtyard-boston-south-boston.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=12&hapos=137&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794530&all_sr_blocks=26519701_94374730_0_2_0&highlighted_blocks=26519701_94374730_0_2_0&matching_block_id=26519701_94374730_0_2_0&sr_pri_blocks=26519701_94374730_0_2_0__67520&from_sustainable_property_sr=1&from=searchresults&map_fdco=1#hotelTmpl
138 https://www.booking.com/hotel/us/stylish-downtown-studio-in-the-boston-cave-34.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&uc

157 https://www.booking.com/hotel/us/renovated-lower-allston-3-bedroom.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=7&hapos=157&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794537&all_sr_blocks=673595601_275552041_6_0_0&highlighted_blocks=673595601_275552041_6_0_0&matching_block_id=673595601_275552041_6_0_0&sr_pri_blocks=673595601_275552041_6_0_0__153270&from=searchresults&map_fdco=1#hotelTmpl
159 https://www.booking.com/hotel/us/brand-new-modern-3bds-2-baths-sleep-9-people-minutes-away-from-boston.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=3041

173 https://www.booking.com/hotel/us/1-cortes-32.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=23&hapos=173&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794537&all_sr_blocks=463442601_310269967_2_0_0&highlighted_blocks=463442601_310269967_2_0_0&matching_block_id=463442601_310269967_2_0_0&sr_pri_blocks=463442601_310269967_2_0_0__107900&from=searchresults&map_fdco=1#hotelTmpl
175 https://www.booking.com/hotel/us/pet-friendly-30min-to-downtown-the-bostonian.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout

188 https://www.booking.com/hotel/us/cozy-private-room-in-coliving-apt.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=13&hapos=188&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794543&all_sr_blocks=792317004_344881626_2_0_0&highlighted_blocks=792317004_344881626_2_0_0&matching_block_id=792317004_344881626_2_0_0&sr_pri_blocks=792317004_344881626_2_0_0__38100&from=searchresults&map_fdco=1#hotelTmpl
189 https://www.booking.com/hotel/us/bright-spacious-private-room-with-shared-bath.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&check

200 https://www.booking.com/hotel/us/hermoso-apartamento-vacacional-en-boston.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout=2023-02-14&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=200&sr_order=popularity&srpvid=b8ac97be9c510304&srepoch=1670794543&all_sr_blocks=884050001_357777868_4_0_0&highlighted_blocks=884050001_357777868_4_0_0&matching_block_id=884050001_357777868_4_0_0&sr_pri_blocks=884050001_357777868_4_0_0__535815&from=searchresults&map_fdco=1#hotelTmpl
201 https://www.booking.com/hotel/us/134-beach-steet.html?label=gen173nr-1FCAEoggI46AdIM1gEaKQCiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuAL5mdmcBsACAdICJGRkMWIwZTJlLTUzNDktNGViYy05NGFlLTUyMjNmMmVhY2U5NdgCBeACAQ&sid=4ccd5a937ba87ec62f771a6b54652f31&aid=304142&ucfs=1&arphpl=1&checkin=2023-02-08&checkout

In [67]:
import os

In [68]:
files = os.listdir('Boston/')

In [69]:
def load_file(filename):
    with open(filename) as f:
        data = json.loads(f.read())
    return data

In [70]:
hotels = [load_file('Boston/' + file) for file in files]

In [71]:
df = pd.DataFrame(hotels)

In [72]:
df.drop('source', axis=1, inplace=True)

In [73]:
df

,hotel_label,location,rating_score,rating_label,total_reviews,best_facility,best_review_score_badge,detail,couple_review,important_facility,price
0,The Godfrey Hotel Boston,"505 Washington Street, Theater District, Bosto...",8.9,Excellent,"5,909 reviews",Excellent location!,9.5,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenitiesvery good Fitness Center...,$905
1,The Lenox,"61 Exeter Street, Back Bay, Boston, MA 02116, ...",9.1,Wonderful,692 reviews,Excellent location!,9.8,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenitiesNon-smoking roomsFitness...,"$1,450"
2,Sonder No 284 Back Bay,"284 Commonwealth Ave., Back Bay, Boston, MA 02...",8.6,Excellent,333 reviews,Excellent location!,9.7,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenitiesNon-smoking roomsFree Wi...,$837
3,Boston Marriott Long Wharf,"296 State Street, Waterfront, Boston, MA 02109...",8.7,Excellent,164 reviews,Excellent location!,9.6,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenities1 swimming poolNon-smoki...,"$1,618"
4,Temple Place Suites 2+3,"29 Temple Place 2nd and 3rd Floor, Theater Dis...",7.6,Good,72 reviews,Excellent location!,9.5,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenitiesNon-smoking roomsFree WiFi,$747
...,...,...,...,...,...,...,...,...,...,...,...
123,AC Hotel by Marriott Boston Downtown,"225 Albany Street, South End, Boston, 02118, U...",8.1,Very Good,189 reviews,Staff,8.6,AC Hotel by Marriott Boston Downtown features ...,Couples in particular like the location – they...,Most popular amenitiesNon-smoking roomsFitness...,"$1,174"
124,Residence Inn Boston Downtown/South End,"2001 Washington Street, South End, Boston, 021...",8.2,Very Good,224 reviews,Staff,8.4,"Featuring a bar, Residence Inn Boston Downtown...",Couples in particular like the location – they...,Most popular amenitiesFitness centerFacilities...,"$1,024"
125,"Downtown Beacon Hill, Convenient, Comfy Studio...","Beacon Hill, Boston, MA 02114, United States o...",7.6,Good,34 reviews,Excellent location!,9.5,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenitiesNon-smoking roomsFree Wi...,$835
126,Clarendon Square,"198 West Brookline Street, South End, Boston, ...",9.6,Exceptional,17 reviews,Staff,9.7,Stay in the heart of Boston–Excellent location...,Couples in particular like the location – they...,Most popular amenitiesNon-smoking roomsFree Wi...,"$1,759"


# need to put the clean code here like remove the \n from both side and all then run last one for getting the data into csv

In [74]:
df.to_csv('Boston.csv')